In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1
import os
import sys
import cv2
import numpy as np
import os

In [3]:
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)
from pipeline import Pipeline

### Set up directory and sample path

In [5]:
# Define path for gallery images
gallery_index_path = "..\\storage\\multi_image_gallery"

# Define path for sample test
probe_folder_path = "..\\simclr_resources\\probe"
sample_probe_name = "Ian_Thorpe\\Ian_Thorpe_0002.jpg"
sample_probe_path = os.path.join(probe_folder_path, sample_probe_name)
sample_probe = Image.open(sample_probe_path)

### Model 1 - VGGFace2

In [6]:
# Initialized pipeline for pre-trained model VGGFace2
pretained1 = 'vggface2'
pipeline1 = Pipeline(pretained1, index_type='brute_force', metric='euclidean')

c:\Users\gsjsc\anaconda3\envs\en605645\Lib\site-packages\facenet_pytorch\models\inception_resnet_v1.py:329: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.

In [7]:
# Define path for faiss index and metadata saving folders
faiss_index_path1 = os.path.join('..', 'storage', f'catalog_{pretained1}')
metadata_path1 = os.path.join('..', 'storage', f'catalog_{pretained1}')

# Precompute and save faiss index and metadata for the model
pipeline1.precompute_and_save(gallery_index_path, faiss_index_path1, metadata_path1)

Precomputed and saved embeddings for the gallery images.


In [8]:
# Load FAISS index and metadata for the model
faiss_index_path1_load = os.path.join(faiss_index_path1, 'faiss_index.bin')
metadata_path1_load = os.path.join(metadata_path1, 'metadata.pkl')

pipeline1.index.load(faiss_index_path1_load, metadata_path1_load)

In [9]:
# Test case for one sample probe
print(f"Under model {pretained1}, for probe {sample_probe_name}, The top 5 nearest neighbors are:\n")
results1 = pipeline1.search_gallery(sample_probe, 5)
for result in results1:
    print(f"Index: {result['index']}, Distance: {result['distance']:.4f}, Name: {result['name']}, Filename: {result['filename']}")

Under model vggface2, for probe Ian_Thorpe\Ian_Thorpe_0002.jpg, The top 5 nearest neighbors are:

Index: 872, Distance: 0.6415, Name: ['Ian_Thorpe'], Filename: ['Ian_Thorpe_0006.jpg']
Index: 1732, Distance: 0.6756, Name: ['Oscar_De_La_Hoya'], Filename: ['Oscar_De_La_Hoya_0003.jpg']
Index: 871, Distance: 0.7435, Name: ['Ian_Thorpe'], Filename: ['Ian_Thorpe_0005.jpg']
Index: 1716, Distance: 0.7961, Name: ['Noah_Wyle'], Filename: ['Noah_Wyle_0001.jpg']
Index: 1500, Distance: 0.8049, Name: ['Mark_Dacey'], Filename: ['Mark_Dacey_0001.jpg']


### Model 2 - casia-webface

In [10]:
# Initialized pipeline for pre-trained model casia-webface
pretained2 = 'casia-webface'
pipeline2 = Pipeline(pretained2, index_type='brute_force', metric='euclidean')

In [11]:
# Define path for gallery images, faiss index and metadata saving folders
faiss_index_path2 = os.path.join('..', 'storage', f'catalog_{pretained2}')
metadata_path2 = os.path.join('..', 'storage', f'catalog_{pretained2}')

# Precompute and save faiss index and metadata for the model
pipeline2.precompute_and_save(gallery_index_path, faiss_index_path2, metadata_path2)

Precomputed and saved embeddings for the gallery images.


In [12]:
# Load FAISS index and metadata for the model
faiss_index_path2_load = os.path.join(faiss_index_path2, 'faiss_index.bin')
metadata_path2_load = os.path.join(metadata_path2, 'metadata.pkl')

pipeline2.index.load(faiss_index_path2_load, metadata_path2_load)

In [13]:
# Check result for the test case
print(f"Under model {pretained2}, for probe {sample_probe_name}, The top 5 nearest neighbors are:\n")
results2 = pipeline2.search_gallery(sample_probe, 5)
for result in results2:
    print(f"Index: {result['index']}, Distance: {result['distance']:.4f}, Name: {result['name']}, Filename: {result['filename']}")

Under model casia-webface, for probe Ian_Thorpe\Ian_Thorpe_0002.jpg, The top 5 nearest neighbors are:

Index: 88, Distance: 0.2395, Name: ['Alvaro_Uribe'], Filename: ['Alvaro_Uribe_0005.jpg']
Index: 642, Distance: 0.2857, Name: ['Gary_Williams'], Filename: ['Gary_Williams_0001.jpg']
Index: 1325, Distance: 0.2909, Name: ['Kim_Ryong-sung'], Filename: ['Kim_Ryong-sung_0001.jpg']
Index: 1150, Distance: 0.2940, Name: ['John_McEnroe'], Filename: ['John_McEnroe_0001.jpg']
Index: 1595, Distance: 0.2980, Name: ['Michael_Phelps'], Filename: ['Michael_Phelps_0003.jpg']


## Model Performance Evaluation

### 1. Mean Reciprocal Rank

In [14]:
# Function to loop over all probes and calculate mean reciprocal rank
def calculate_mrr(probe_folder_path, pipeline, k=5):

    reciprocal_rank = 0
    total_probe = 0

    for probe_name in os.listdir(probe_folder_path):
        probe_person_folder = os.path.join(probe_folder_path, probe_name)

        for probe_file_name in os.listdir(probe_person_folder):
            if probe_file_name.endswith(('.jpg', '.png', '.jpeg')) and not probe_file_name.startswith('._'):
                image_path = os.path.join(probe_person_folder, probe_file_name)
                total_probe += 1

                try:
                    with Image.open(image_path) as img:

                        # 1. Get the top neighbors for the probe
                        results = pipeline.search_gallery(img, k)

                        #2. Check if the probe name are in the returned relevant items
                        names = [result['name'] for result in results]
                        for i in range(k):
                            if probe_name == names[i][0]:
                                reciprocal_rank += 1/(i+1)
                                break

                except PIL.UnidentifiedImageError:
                    print(f"Skipping file {image_path}: UnidentifiedImageError")
    
    mrr = reciprocal_rank/total_probe * 100
    print(f"Mean Recriprocal Rank: {mrr:.2f}%")

In [16]:
probe_folder_path = "..\\simclr_resources\\probe"
print(f"For model {pretained1}:")
calculate_mrr(probe_folder_path, pipeline1, k=5)

For model vggface2:
Mean Recriprocal Rank: 56.58%


In [17]:
print(f"For model {pretained2}:")
calculate_mrr(probe_folder_path, pipeline2, k=5)

For model casia-webface:
Mean Recriprocal Rank: 11.77%


### 2. Precision@k

In [18]:
# Function to loop over all probes and calculate precision@k
def precision_k(probe_folder_path, pipeline, k=5):

    total_precision = 0
    total_probe = 0

    for probe_name in os.listdir(probe_folder_path):
        probe_person_folder = os.path.join(probe_folder_path, probe_name)

        for probe_file_name in os.listdir(probe_person_folder):
            if probe_file_name.endswith(('.jpg', '.png', '.jpeg')) and not probe_file_name.startswith('._'):
                image_path = os.path.join(probe_person_folder, probe_file_name)
                total_probe += 1

                try:
                    with Image.open(image_path) as img:

                        # 1. Get the top neighbors for the probe
                        results = pipeline.search_gallery(img, k)

                        # 2. Check if the probe name are in the returned relevant items
                        relevant = 0

                        names = [result['name'] for result in results]
                        for i in range(k):
                            if probe_name == names[i][0]:
                                relevant += 1
                        
                        # 3. Calculate precision
                        precision = relevant/k
                        total_precision += precision

                except PIL.UnidentifiedImageError:
                    print(f"Skipping file {image_path}: UnidentifiedImageError")
    
    mrr = total_precision/total_probe
    print(f"Precision@k when k = {k}: {mrr:.2f}")

In [19]:
print(f"For model {pretained1}:")
precision_k(probe_folder_path, pipeline1, k=5)

For model vggface2:
Precision@k when k = 5: 0.21


In [37]:
print(f"For model {pretained2}:")
precision_k(probe_folder_path, pipeline2, k=5)

For model casia-webface:
Precision@k when k = 5: 0.04


## Adding Noise Transformations

In [20]:
# Flip the image horizontally
def horizontal_flip(image):
    return cv2.flip(image, 1)

# Apply a Gaussian blur to smooth the image
def gaussian_blur(image, kernel_size=(5, 5), sigma=0):
    return cv2.GaussianBlur(image, kernel_size, sigma)

# Resize the image to the specified size
def resize(image, new_size):
    return cv2.resize(image, new_size)

# Crop the image to the specified size
def random_crop(image, crop_size):
    h, w = image.shape[:2]
    crop_h, crop_w = crop_size
    start_y = np.random.randint(0, h - crop_h + 1)
    start_x = np.random.randint(0, w - crop_w + 1)
    return image[start_y:start_y + crop_h, start_x:start_x + crop_w]

# Rotate the image by a given angle
def rotate(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, M, (w, h))

# Increase or decrease the brightness of the image
def adjust_brightness(image, value=30):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v = cv2.add(v, value)
    v = np.clip(v, 0, 255)
    final_hsv = cv2.merge((h, s, v))
    return cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)


In [35]:
def apply_noise_transformations(image):
    """
    Apply various noise transformations to an image and return a dictionary 
    containing the transformed images.
    """
    # Define various severity levels for transformations
    severity_levels = {
        'blur': [(15, 15), (31, 31)],  # kernel sizes for Gaussian blur
        'brightness': [60, -60],     # brightness increase and decrease
        'rotate': [15, 45],     # degrees to rotate
        'crop_size': [(70, 70), (100, 100)]  # cropping sizes
    }
    
    # Store all transformed images in a dictionary
    transformed_images = {
        'original': image,
        'horizontal_flip': horizontal_flip(image),
        'gaussian_blur_light': gaussian_blur(image, severity_levels['blur'][0]),
        'gaussian_blur_strong': gaussian_blur(image, severity_levels['blur'][1]),
        'resize_small': resize(image, (128, 128)),
        'resize_large': resize(image, (256, 256)),
        'random_crop_small': random_crop(image, severity_levels['crop_size'][0]),
        'random_crop_large': random_crop(image, severity_levels['crop_size'][1]),
        'rotate_small': rotate(image, severity_levels['rotate'][0]),
        'rotate_large': rotate(image, severity_levels['rotate'][1]),
        'brightness_increase': adjust_brightness(image, severity_levels['brightness'][0]),
        'brightness_decrease': adjust_brightness(image, severity_levels['brightness'][1])
    }
    
    return transformed_images

In [36]:
def evaluate_transformed_images_mrr(probe_folder_path, pipeline, k=5):
    """
    Evaluate the Mean Reciprocal Rank (MRR) performance impacts of various noise transformations 
    on probe images.
    """
    total_mrr = {key: 0 for key in ['original', 'horizontal_flip', 'gaussian_blur_light', 'gaussian_blur_strong',
                                    'resize_small', 'resize_large', 'random_crop_small', 'random_crop_large',
                                    'rotate_small', 'rotate_large', 'brightness_increase', 'brightness_decrease']}
    
    total_probes = 0

    for probe_name in os.listdir(probe_folder_path):
        probe_person_folder = os.path.join(probe_folder_path, probe_name)

        for probe_file_name in os.listdir(probe_person_folder):
            if probe_file_name.endswith(('.jpg', '.png', '.jpeg')) and not probe_file_name.startswith('._'):
                image_path = os.path.join(probe_person_folder, probe_file_name)
                total_probes += 1

                try:
                    # Open the image
                    probe_image = Image.open(image_path)
                    probe_image_cv2 = np.array(probe_image)

                    # Apply noise transformations
                    transformed_images = apply_noise_transformations(probe_image_cv2)

                    # Evaluate for each transformation
                    for transform_name, transformed_image in transformed_images.items():
                        transformed_image_pil = Image.fromarray(transformed_image)
                        
                        # Get the top neighbors for the transformed probe image
                        results = pipeline.search_gallery(transformed_image_pil, k)

                        # Calculate Reciprocal Rank for the transformation
                        names = [result['name'] for result in results]
                        reciprocal_rank = 0

                        for i in range(k):
                            if probe_name == names[i][0]:
                                reciprocal_rank = 1 / (i + 1)
                                break 
                        
                        # Add reciprocal rank to total
                        total_mrr[transform_name] += reciprocal_rank

                except PIL.UnidentifiedImageError:
                    print(f"Skipping file {image_path}: UnidentifiedImageError")

    # Calculate the average MRR for each transformation
    for transform_name in total_mrr:
        avg_mrr = total_mrr[transform_name] / total_probes
        print(f"Transformation: {transform_name} | Mean Reciprocal Rank (MRR): {avg_mrr:.2f}")


In [37]:
evaluate_transformed_images_mrr(probe_folder_path, pipeline1, k=5)

Transformation: original | Mean Reciprocal Rank (MRR): 0.57
Transformation: horizontal_flip | Mean Reciprocal Rank (MRR): 0.56
Transformation: gaussian_blur_light | Mean Reciprocal Rank (MRR): 0.44
Transformation: gaussian_blur_strong | Mean Reciprocal Rank (MRR): 0.09
Transformation: resize_small | Mean Reciprocal Rank (MRR): 0.57
Transformation: resize_large | Mean Reciprocal Rank (MRR): 0.57
Transformation: random_crop_small | Mean Reciprocal Rank (MRR): 0.02
Transformation: random_crop_large | Mean Reciprocal Rank (MRR): 0.12
Transformation: rotate_small | Mean Reciprocal Rank (MRR): 0.48
Transformation: rotate_large | Mean Reciprocal Rank (MRR): 0.04
Transformation: brightness_increase | Mean Reciprocal Rank (MRR): 0.28
Transformation: brightness_decrease | Mean Reciprocal Rank (MRR): 0.20


**Summary of Effects:**
The system shows minimal sensitivity to horizontal flips and resizing, with slight degradation under small rotations and light blur. However, strong blur, large rotations, and significant brightness changes cause notable performance drops, especially in random crops, which severely impair accuracy.

**Design Thoughts:**
To improve robustness, the system should incorporate image preprocessing steps like face detection, alignment, and brightness normalization. Training with augmented data that includes small rotations, light blurs, and brightness variations can make the model more resilient. For extreme cases (e.g., strong blur or large rotation), the system could trigger a re-capture mechanism to ensure image quality.